https://www.kaggle.com/c/state-farm-distracted-driver-detection/overview

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import trange

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Data Preprocessing

In [3]:
X = []
y = []
for i in range(10):
    dir_path =f'data/imgs/train/c{i}'
    files = os.listdir(dir_path)
    for img_name in files[:120]:
        full_path = dir_path + '/' + img_name
        # print(full_path)
        img = np.array(Image.open(full_path))
        # print(img.shape)
        X.append(img)
        y.append(i)

X = torch.tensor(np.array(X)).float().view(-1, 3, 640, 480)
y = torch.tensor(np.array(y)).float()
X.shape, y.shape

(torch.Size([1200, 3, 640, 480]), torch.Size([1200]))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
class DriverDataset(Dataset):
    def __init__(self, x_, y_):
        self.x_ = x_.to(device)
        self.y_ = y_.to(device)
    
    def __len__(self):
        return self.x_.shape[0]
    
    def __getitem__(self, index):
        return self.x_[index], self.y_[index]

In [6]:
train_dataset = DriverDataset(X_train, y_train)
test_dataset = DriverDataset(X_test, y_test)

train_loader = DataLoader(train_dataset,batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=128, shuffle=False)

## Model

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.l1 = nn.Linear(16*158*118,640)
        self.l2 = nn.Linear(640, 160)
        self.l3 = nn.Linear(160, 10)
    
    def forward(self, x):
        # x -> (n, 3, 640, 480)
        out = self.pool(F.relu(self.conv1(x))) # -> (n, 6, 318, 238)
        out = self.pool(F.relu(self.conv2(out))) # -> (n, 16, 158, 118)
        out = out.view(-1, 16*158*118)
        out = F.relu(self.l1(out))
        out = F.relu(self.l2(out))
        out = self.l3(out)
        return out

model = CNN().to(device)

In [8]:
loss_category = nn.CrossEntropyLoss().to(device)
optimiser = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [9]:
model = torch.compile(model)

In [10]:
torch.set_float32_matmul_precision('high')

In [14]:
for epoch in trange(20):
    for i, (x_, y_) in enumerate(train_loader):
        x_ = x_.to(torch.float32).to(device)
        y_ = F.one_hot(y_.clone().detach().long(), 10).float().to(device)
        y_pred = model(x_)
        loss = loss_category(y_pred, y_)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()

    if (epoch+1)%5 == 0: print(f'Epoch {epoch+1}: {loss.item():3f}')

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 5: 2.497697
Epoch 10: 0.836714
Epoch 15: 0.041116
Epoch 20: 0.010473


In [15]:
model.eval()
score = 0
tot = 0
with torch.no_grad():
    for i, (x_, y_) in enumerate(train_loader):
        x_ = x_.to(torch.float32).to(device)
        y_pred = model(x_)
        for y1, y_pred1 in zip(y_.int(), torch.argmax(y_pred, dim=1)):
            if y1.item() == y_pred1.item(): score += 1
            tot += 1
print('Train acc:', score*100/ tot, '%')

Train acc: 99.88888888888889 %


In [16]:
model.eval()
score = 0
tot = 0
with torch.no_grad():
    for i, (x_, y_) in enumerate(test_loader):
        x_ = x_.to(torch.float32).to(device)
        y_pred = model(x_)
        for y1, y_pred1 in zip(y_.int(), torch.argmax(y_pred, dim=1)):
            if y1.item() == y_pred1.item(): score += 1
            tot += 1
print('Test acc:', score*100/ tot, '%')


Test acc: 74.0 %


Best acc: 75.6 % 